In [13]:
import time
from selenium import webdriver
import sqlite3
from datetime import datetime as dt
import os
from pathlib import Path
import sys
import json

In [55]:
class Report():
    def __init__(self):
        self.name = ""
        self.deadline = ""
        self.filename = ""
        self.reportname = ""
        
    def subid2name(self, subid):
        comm = f"select * from subs where subid = {subid}"
        i = tuple(c.execute(comm))
        return i[0][1]

class Task():
    def __init__(self):
        self.path = ""
        self.subject = ""
        self.deadline = 0
        self.deadline_str = ""
        self.task_hash = ""
        
    def id_gen(self, data):
        return int(data)+1

    def deadline_gen(self, dl_str):
        dl_pre_int = dt.strptime(dl_str, "%Y-%m-%d %H:%M:%S")
        dl_int = dl_pre_int.timestamp()
        return dl_int

def getLatestDownloadedFileName(arg):
    p = Path(arg)
    files = list(p.glob("*"))
    file_updates = {file_path: os.stat(file_path).st_mtime for file_path in files}

    newst_file_path = max(file_updates, key=file_updates.get)
    return newst_file_path


def dup_check(filename):
    comm = f"select * from files where file='{filename}'"
    if len(tuple(c.execute(comm))) == 0:
        c.execute(f'insert into files values("{filename}")')
        return False
    else:
        return True

chromeOptions = webdriver.ChromeOptions()
prefs = {"download.default_directory" : r"D:\Users\maple\OneDrive - keio.jp\★課題★\.temp"}
chromeOptions.add_experimental_option("prefs",prefs)

In [92]:
conn = sqlite3.connect('subs_2020.db')
c = conn.cursor()

In [7]:
mail = "maple_1031@keio.jp"
password = "keio1031"

chrome_path = "D:\\Users\\maple\\Python\\chromedriver_win32\\chromedriver.exe"
driver = webdriver.Chrome(executable_path=chrome_path, options=chromeOptions)
driver.get('https://portal.keio.jp/koid/')
time.sleep(1)
search_box = driver.find_element_by_id("username")
search_box.send_keys(mail)
pass_box = driver.find_element_by_id("password")
pass_box.send_keys(password)
submit_bottun = driver.find_element_by_xpath("/html/body/div/div[1]/div/div/form/section[3]/button")
submit_bottun.click()
time.sleep(0.5)

seminor_botton = driver.find_element_by_id("ui-id-12")
seminor_botton.click()

seminor_suport_button = driver.find_element_by_id("ui-id-51")
seminor_suport_button.click()

driver.switch_to.window(driver.window_handles[1])
time.sleep(0.5)

driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(0.5)

report_button = driver.find_elements_by_xpath("//form[@action=\"/ess2/reportAction\"]/button")
report_subid = driver.find_elements_by_xpath("//form[@action=\"/ess2/reportAction\"]/input[@name=\"SELECT_ENTNO\"]")

In [95]:
report_button = driver.find_elements_by_xpath("//form[@action=\"/ess2/reportAction\"]/button")
report_subid = driver.find_elements_by_xpath("//form[@action=\"/ess2/reportAction\"]/input[@name=\"SELECT_ENTNO\"]")

report_list = []

for i in range(len(report_button)):
    report_temp = []
    report_button = driver.find_elements_by_xpath("//form[@action=\"/ess2/reportAction\"]/button")
    report_subid = driver.find_elements_by_xpath("//form[@action=\"/ess2/reportAction\"]/input[@name=\"SELECT_ENTNO\"]")
    report = Report()
    argid = int(report_subid[i].get_attribute("value"))
    report.name = report.subid2name(argid)
    report_button[i].click()
    report.deadline = driver.find_element_by_xpath('//*[@id="student"]/div[2]/div[1]/form/div[1]/div/div/div/div[2]/div[5]/div/p').text
    temp = driver.find_element_by_xpath('//*[@id="student"]/div[2]/div[1]/form/div[1]/div/div/div/div[2]/div[1]/div/p').text

    
    if not dup_check(temp):
        try:
            driver.find_element_by_xpath('//*[@name="BTN__DL"]')
        except:
            report.filename = "#"
        else:
            driver.find_element_by_xpath('//*[@name="BTN__DL"]').click()
            try:
                driver.find_element_by_xpath('//button[@onclick="agreeSubmit("#_DL", "");"]').click()
            except:
                pass
            else:
                pass

            timeout_second = 5

            for i in range(timeout_second + 1):
                download_fileName = glob.glob(f'{downloadsFilePath}\\*.*')

                if download_fileName:
                    extension = os.path.splitext(download_fileName[0])

                    if '.crdownload' not in extension[1] : break

                if i >= timeout_second:
                    driver.quit()
                    shutil.rmtree(tmp_download_dir)
                    sys.exit()

                time.sleep(1)

            report.filename = getLatestDownloadedFileName(downloadsFilePath)
    report.reportname = temp
    
    report_temp = [report.name, report.deadline, report.reportname, report.filename]
    report_list.append(report_temp)
    
    driver.find_element_by_xpath('//button[@name="BTN_JIKANWARI"]').click()
    
print(report_list)

[['応用解析第２', '2020-12-24 18:00:00', '応用解析第２試験解答（１０時４５分以降１８時まで）', ''], ['応用解析第２', '2020-12-24 10:45:00', '応用解析第２試験解答（１０時４５分まで）', ''], ['応用解析第２', '2020-12-19 23:50:00', 'レポート３', ''], ['応用解析第２', '2020-12-05 23:50:00', 'レポート２', ''], ['創造と倫理', '2020-12-31 00:00:00', 'やむを得ず欠席してしまった学生向けレポート', ''], ['創造と倫理', '2020-12-28 23:59:00', '最終レポート', ''], ['統計解析の基礎', '2020-10-31 00:00:00', 'Tentative', '']]


In [99]:
hash("".join(report_list[0][:3]))

2290121610131828602

In [237]:
conn = sqlite3.connect('current_tasks.db')

c = conn.cursor()

In [88]:
downloadsFilePath = r'D:\Users\maple\OneDrive - keio.jp\★課題★\.temp'

for i in report_list:
    task = Task()
    with open("current_tasks.json") as f:
        df = json.load(f)

    task.path = i[3]
    task.subject = i[0]
    task.deadline = int(task.deadline_gen(i[1]))
    task.deadline_str = i[1]
    task.task_hash = hash("".join(i[:3]))
    temp_dict = {"path":task.path, "subject":task.subject, "deadline_unix":task.deadline, "deadline_str":task.deadline_str, "task_hash":task.task_hash}
    if temp_dict not in list(df.values())[0]:
        df["current_tasks"].append(temp_dict)

    with open("current_tasks.json", "w") as f:
        json.dump(df, f, indent=4)

In [102]:
conn.close()

In [249]:
driver.find_element_by_xpath('//*[@id="student"]/div[2]/div[1]/form/div[1]/div/div/div/div[2]/div[1]/div/p')

<selenium.webdriver.remote.webelement.WebElement (session="630f59a9f1ba3735c86bd869b4e124b8", element="472e18c1-d200-4dfe-9ab3-2b12c074e160")>

In [100]:
for i in c.execute("select * from files"):
    print(i)

('応用解析第２試験解答（１０時４５分以降１８時まで）',)
('応用解析第２試験解答（１０時４５分まで）',)
('レポート３',)
('レポート２',)
('やむを得ず欠席してしまった学生向けレポート',)
('最終レポート',)
('Tentative',)


In [326]:
c.execute("insert into files values('aaa')")

In [50]:
c.execute("delete from files")

In [75]:
list(df.values())[0][0]

{'path': '#',
 'subject': '応用解析第２',
 'deadline_unix': 1608800400,
 'deadline_str': '2020-12-24 18:00:00',
 'task_hash': 2290121610131828602}

In [104]:
with open("current_tasks.json", mode="rt", encoding="utf-8") as f:
    df = json.load(f)

In [105]:
print(df)

{'current_tasks': [{'path': '', 'subject': '応用解析第２', 'deadline_unix': 1608800400, 'deadline_str': '2020-12-24 18:00:00', 'deadline_day_str': '2020-12-24', 'task_hash': 5475273046561075995}, {'path': '', 'subject': '応用解析第２', 'deadline_unix': 1608774300, 'deadline_str': '2020-12-24 10:45:00', 'deadline_day_str': '2020-12-24', 'task_hash': -4925489119067566978}, {'path': '', 'subject': '応用解析第２', 'deadline_unix': 1608389400, 'deadline_str': '2020-12-19 23:50:00', 'deadline_day_str': '2020-12-19', 'task_hash': 4490378651875545470}, {'path': '', 'subject': '応用解析第２', 'deadline_unix': 1607179800, 'deadline_str': '2020-12-05 23:50:00', 'deadline_day_str': '2020-12-05', 'task_hash': -7783411213281798483}, {'path': '', 'subject': '創造と倫理', 'deadline_unix': 1609340400, 'deadline_str': '2020-12-31 00:00:00', 'deadline_day_str': '2020-12-31', 'task_hash': -4685833743494414334}, {'path': '', 'subject': '創造と倫理', 'deadline_unix': 1609167540, 'deadline_str': '2020-12-28 23:59:00', 'deadline_day_str': '20

In [108]:
print(len(df["current_tasks"]))

7


In [111]:
now = dt.datetime.now(dt.timezone(dt.timedelta(hours=9)))

AttributeError: type object 'datetime.datetime' has no attribute 'datetime'

In [112]:
dt.now

<function datetime.now(tz=None)>

In [113]:
print(dt.now)

<built-in method now of type object at 0x00007FF9C213B530>


In [114]:
dt.now.strftime('%Y-%m-%d %H:%M:%S')

AttributeError: 'builtin_function_or_method' object has no attribute 'strftime'